In [1]:
import chalenge1000
import model_lib
import collections
from IPython.display import display, HTML
import pandas


Chalenge = chalenge1000.Native()
df = Chalenge.X

In [2]:
def vocab(element):
    if type(element) == str:
        return ' '.join(set(element.split(' ')))
    else:
        return ''

def _agriculture__(element):
    if element in agriculture:
        return True
    else:
        return False
    
col_count = lambda x: ' '.join([a for a,b in collections.Counter(x.split()).most_common(10000)])
lemm = lambda x: model_lib.Lemm(x,file = 'stopwords_agri.txt').X

def df_fmt(df):
    df['corpus'] = df.join(Chalenge.descriptions_trad)[['prez_struc','prez_produit_struc']].agg(sum, axis = 1)
    df['type'] = df['corpus'].map(type)
    df['corpus'] = df['corpus'].map(vocab)
    df['corpus_lemm'] = df['corpus'].map(lemm)
    df['collection_counter'] = df['corpus_lemm'].map(col_count)
    agr = {e for e in df['cat_autre_struc'].dropna().values.tolist() if 'agr' in e}
    alim = {e for e in df['cat_autre_struc'].dropna().values.tolist() if 'alim' in e}
    agriculture = alim.union(agr)
    df = df[df['type'] ==str]
    return df

def get_topic(e):
    if e >0:
        return 'agriculture'
    if e<0:
        return 'media'
    
def true_positve(a,b):
    if a == b:
        return 1
    else:
        return 0

In [3]:
df = df_fmt(df)

# Entrainement
## LDA

In [15]:

#model = model_lib.Models(n_components = n_components, n_top_words =n_top_words, n_features = n_features)
def run_model_lda(model_parameters,text_corpus):
    def get_topic(e):
        if e >0:
            return 'agriculture'
        if e<0:
            return 'media'
    
    def true_positve(a,b):
        if a == b:
            return 1
        else:
            return 0
    model = model_lib.Models(**model_parameters)

    models = model.run_model_LDA(text_corpus.values.tolist())

    predict = pandas.DataFrame(model.reverse_lda(text_corpus.values.tolist(), models) , index =text_corpus.index)
    display(predict)

    predict = predict.join(df['categorie'])
    predict[0] = predict[0]*100
    predict[1] = predict[1]*100
    predict['topic'] = predict[0]-predict[1]
    display(predict)

    predict['predict_label'] = predict['topic'].map(get_topic)
    display(predict)

    predict['classification_kpi'] = predict.apply(lambda x: true_positve(x['categorie'],x['predict_label']), axis= 1)
    display(predict)

    agri = predict[predict['categorie'] == 'agriculture']
    print(model_parameters)
    print('precision {}'.format(agri['classification_kpi'].sum()/695*100))
    errors = agri[agri['classification_kpi'] == 0].sort_values('topic')
    collections.Counter(' '.join(errors.join(text_corpus)['collection_counter'].values.tolist()).split()).most_common(10)

In [23]:
text_corpus = df[df['categorie'].isin(['agriculture','media'])]['collection_counter']
list_model_parameters_prior = [{'n_components' : 2, 'n_top_words' : 20, 'n_features' : 20000, 'doc_topic_prior':alpha/10} for alpha in range(1,10,1)]

for model_parameters in list_model_parameters_prior:
    run_model_lda(model_parameters,text_corpus)

Extracting tf features for LDA...
done in 0.185s.

Fitting LDA models with tf features, n_samples=2000 and n_features=20000...
done in 0.922s.

Topics in LDA model:
Topic #0: product application market smartphone agricultural production basketballbasketball create rural service clinicagro playstore produce promagric web farmersbreeders malapse farmer instantly platform
Topic #1: product production market make agricultural create produce food farmer service local farm agriculture young woman sustainable rural management develop order



,0,1
key_main,,
http://chapwifi.bj/_adjassa_azimath,0.002370,0.997630
https://www.facebook.com/TZenSn/_diop_sokhnalaye,0.003969,0.996031
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,0.001330,0.998670
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,0.003012,0.996988
http://www.retpaci.ci_kouassi_ayagazalo,0.001147,0.998853
...,...,...
http://www.cbdibaong.org_houedanou_martin,0.002315,0.997685
http://www.cbdibaong.org_yemadje_emeline,0.004717,0.995283
http://www.cbdibaong.org_akokponsode_hippolyte,0.002119,0.997881


,0,1,categorie,topic
key_main,,,,
http://chapwifi.bj/_adjassa_azimath,0.237016,99.762984,media,-99.525968
https://www.facebook.com/TZenSn/_diop_sokhnalaye,0.396876,99.603124,agriculture,-99.206247
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,0.132995,99.867005,agriculture,-99.734010
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,0.301242,99.698758,agriculture,-99.397517
http://www.retpaci.ci_kouassi_ayagazalo,0.114694,99.885306,agriculture,-99.770612
...,...,...,...,...
http://www.cbdibaong.org_houedanou_martin,0.231495,99.768505,agriculture,-99.537011
http://www.cbdibaong.org_yemadje_emeline,0.471728,99.528272,agriculture,-99.056544
http://www.cbdibaong.org_akokponsode_hippolyte,0.211876,99.788124,agriculture,-99.576248


,0,1,categorie,topic,predict_label
key_main,,,,,
http://chapwifi.bj/_adjassa_azimath,0.237016,99.762984,media,-99.525968,media
https://www.facebook.com/TZenSn/_diop_sokhnalaye,0.396876,99.603124,agriculture,-99.206247,media
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,0.132995,99.867005,agriculture,-99.734010,media
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,0.301242,99.698758,agriculture,-99.397517,media
http://www.retpaci.ci_kouassi_ayagazalo,0.114694,99.885306,agriculture,-99.770612,media
...,...,...,...,...,...
http://www.cbdibaong.org_houedanou_martin,0.231495,99.768505,agriculture,-99.537011,media
http://www.cbdibaong.org_yemadje_emeline,0.471728,99.528272,agriculture,-99.056544,media
http://www.cbdibaong.org_akokponsode_hippolyte,0.211876,99.788124,agriculture,-99.576248,media


,0,1,categorie,topic,predict_label,classification_kpi
key_main,,,,,,
http://chapwifi.bj/_adjassa_azimath,0.237016,99.762984,media,-99.525968,media,1
https://www.facebook.com/TZenSn/_diop_sokhnalaye,0.396876,99.603124,agriculture,-99.206247,media,0
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,0.132995,99.867005,agriculture,-99.734010,media,0
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,0.301242,99.698758,agriculture,-99.397517,media,0
http://www.retpaci.ci_kouassi_ayagazalo,0.114694,99.885306,agriculture,-99.770612,media,0
...,...,...,...,...,...,...
http://www.cbdibaong.org_houedanou_martin,0.231495,99.768505,agriculture,-99.537011,media,0
http://www.cbdibaong.org_yemadje_emeline,0.471728,99.528272,agriculture,-99.056544,media,0
http://www.cbdibaong.org_akokponsode_hippolyte,0.211876,99.788124,agriculture,-99.576248,media,0


...

,0,1
key_main,,
http://chapwifi.bj/_adjassa_azimath,0.075767,0.924233
https://www.facebook.com/TZenSn/_diop_sokhnalaye,0.052028,0.947972
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,0.017406,0.982594
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,0.039130,0.960870
http://www.retpaci.ci_kouassi_ayagazalo,0.015078,0.984922
...,...,...
http://www.cbdibaong.org_houedanou_martin,0.024320,0.975680
http://www.cbdibaong.org_yemadje_emeline,0.069279,0.930721
http://www.cbdibaong.org_akokponsode_hippolyte,0.023028,0.976972


,0,1,categorie,topic
key_main,,,,
http://chapwifi.bj/_adjassa_azimath,7.576688,92.423312,media,-84.846625
https://www.facebook.com/TZenSn/_diop_sokhnalaye,5.202821,94.797179,agriculture,-89.594357
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,1.740565,98.259435,agriculture,-96.518870
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,3.913016,96.086984,agriculture,-92.173968
http://www.retpaci.ci_kouassi_ayagazalo,1.507769,98.492231,agriculture,-96.984462
...,...,...,...,...
http://www.cbdibaong.org_houedanou_martin,2.432012,97.567988,agriculture,-95.135975
http://www.cbdibaong.org_yemadje_emeline,6.927897,93.072103,agriculture,-86.144206
http://www.cbdibaong.org_akokponsode_hippolyte,2.302761,97.697239,agriculture,-95.394478


,0,1,categorie,topic,predict_label
key_main,,,,,
http://chapwifi.bj/_adjassa_azimath,7.576688,92.423312,media,-84.846625,media
https://www.facebook.com/TZenSn/_diop_sokhnalaye,5.202821,94.797179,agriculture,-89.594357,media
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,1.740565,98.259435,agriculture,-96.518870,media
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,3.913016,96.086984,agriculture,-92.173968,media
http://www.retpaci.ci_kouassi_ayagazalo,1.507769,98.492231,agriculture,-96.984462,media
...,...,...,...,...,...
http://www.cbdibaong.org_houedanou_martin,2.432012,97.567988,agriculture,-95.135975,media
http://www.cbdibaong.org_yemadje_emeline,6.927897,93.072103,agriculture,-86.144206,media
http://www.cbdibaong.org_akokponsode_hippolyte,2.302761,97.697239,agriculture,-95.394478,media


,0,1,categorie,topic,predict_label,classification_kpi
key_main,,,,,,
http://chapwifi.bj/_adjassa_azimath,7.576688,92.423312,media,-84.846625,media,1
https://www.facebook.com/TZenSn/_diop_sokhnalaye,5.202821,94.797179,agriculture,-89.594357,media,0
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,1.740565,98.259435,agriculture,-96.518870,media,0
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,3.913016,96.086984,agriculture,-92.173968,media,0
http://www.retpaci.ci_kouassi_ayagazalo,1.507769,98.492231,agriculture,-96.984462,media,0
...,...,...,...,...,...,...
http://www.cbdibaong.org_houedanou_martin,2.432012,97.567988,agriculture,-95.135975,media,0
http://www.cbdibaong.org_yemadje_emeline,6.927897,93.072103,agriculture,-86.144206,media,0
http://www.cbdibaong.org_akokponsode_hippolyte,2.302761,97.697239,agriculture,-95.394478,media,0


{'n_components': 2, 'n_top_words': 20, 'n_features': 20000, 'doc_topic_prior': 0.9}
precision 1.7266187050359711


In [6]:
template = errors.join(chalenge1000.Native().text['temp'])['temp'].values.tolist()
with open('misclassified_agri.txt', 'w') as file:
    for line in template:
        file.writelines(line)
        file.writelines('\n')
        file.writelines('##################\n')

In [13]:
print(len(errors))
print(set(errors.join(Chalenge.descriptions, lsuffix = 'left')['categorie']))

raw = pandas.read_csv(chalenge1000.RefEnv().transformed.format('df1_clean.csv'))
set(raw[raw.index.isin(errors.index)]['cat_struc'])

690
{'agriculture'}


set()

# catégorie abscente

In [23]:
label = pandas.read_csv('datastore.csv').set_index('key_main')
label['corpus'] = label[label['_agriculture_']][['prez_struc','prez_produit_struc']].agg(sum, axis = 1)
label['type'] = label['corpus'].map(type)
label['corpus'] = label['corpus'].map(vocab)
label['corpus_lemm'] = label['corpus'].map(lemm)
label['collection_counter'] = label['corpus_lemm'].map(col_count)
label = label[label['type'] ==str]

n_components = 2
n_top_words = 20
n_features = 20000
text_corpus = label['collection_counter']

predict = pandas.DataFrame(model.reverse_lda(text_corpus.values.tolist(), models) , index =text_corpus.index)
predict = predict.join(df['categorie'])
predict[0] = predict[0]*100
predict[1] = predict[1]*100
predict['topic'] = predict[0]-predict[1]
predict['predict_label'] = predict['topic'].map(get_topic)
predict['classification_kpi'] = predict.apply(lambda x: true_positve(x['categorie'],x['predict_label']), axis= 1)
predict.sort_values(0, ascending = False)[[0,1,'predict_label','topic']]

,0,1,predict_label,topic
key_main,,,,
http://www.kobo-hub.com_latere_atiyasidonie,79.930454,20.069546,agriculture,59.860908
https://www.petitepirogue.com_matembe_herve,78.480356,21.519644,agriculture,56.960713
http://YamAgroIndustrie.bf_yameogo_stevearmandleonardwendkouni,76.478304,23.521696,agriculture,52.956608
http://www.senebusiness.com_sene_ablaye,73.139760,26.860240,agriculture,46.279521
https://www.youtube.com/results?search_query=gue+douade+marius+biographie_gue_douademarius,66.575119,33.424881,agriculture,33.150239
http://www.madeintogo.com_wattara_fankjah,64.386611,35.613389,agriculture,28.773223
http://www.congofresh.com_enge_albert,64.193171,35.806829,agriculture,28.386341
https://web.facebook.com/davenbakery_ongmessom_blanche,59.225808,40.774192,agriculture,18.451615
https://4piment.blogspot.com/_sinare_idrissa,59.044767,40.955233,agriculture,18.089534


In [47]:
def print_desc(already_read = set()):
    import random

    #predict_table = predict[(predict['predict_label'] == 'agriculture')&(predict['categorie'] != 'agriculture')]
    predict_table = predict.join(df['cat_autre_struc'].map(_agriculture__)).join(df['nom_struc']) 
    descriptions = Chalenge.descriptions_trad

    set_index = set(range(len(predict_table)))
    i = random.choice(list(set_index.difference(already_read)))
    already_read = already_read.add(i)
    print('nombre de records', len(predict_table))
    print('index n°', i, '\n')

    categorie,topic,predict_label,prez_struc,prez_produit_struc, nom_struc= predict_table.join(descriptions)[['categorie','topic','predict_label','prez_struc','prez_produit_struc', 'nom_struc']].values.tolist()[i]
    
    print('categorie initiliale:', categorie)
    print('label lda:', predict_label, '\n')
    print('confiance lda:',abs(topic), '\n')
    print(nom_struc, '\n')
    print(prez_struc, '\n')
    print(prez_produit_struc, '\n')
    return already_read
   
print_desc()
predict.head()

nombre de records 15
index n° 1 

categorie initiliale: autres
label lda: media 

confiance lda: 28.79098151101499 

Aquareign industry 

In August 2016, Aquareign industry, called Aqua'frica product, started its activities as a service provider in the coastal, South, Centre, West, North, East and Far North regions of Cameroon where it provides 45% of the services of the current total.
 From 2017 it has added to its catalogue of activities: innovation, production of table fish and fry, fish processing, training/internships and agro-tourism.
Especially in July 2017, Aqua'frica product is transformed into Aqua'frica consulting and becomes a legal company registered in the trade register under the number RC/YAO/2017/A/2973 and taxpayer number P049312632432N .  
 Since 2018, Aquareign industry is planning to set up 100 SFE project kits in urban and rural areas and to fully install two large farms (aquaculture and agro-ecological) thanks to a combined participation of autonomous financing a

,0,1,categorie,topic,predict_label,classification_kpi
key_main,,,,,,
http://YamAgroIndustrie.bf_yameogo_stevearmandleonardwendkouni,76.478304,23.521696,autres,52.956608,agriculture,0
http://www.aquareignindustry.com_mbouleediango_marcidriss,35.604509,64.395491,autres,-28.790982,media,0
http://www.congofresh.com_enge_albert,64.193171,35.806829,autres,28.386341,agriculture,0
http://www.kobo-hub.com_latere_atiyasidonie,79.930454,20.069546,autres,59.860908,agriculture,0
http://www.leader-manioc-96.websel.net/site%20map.xml_adognon_alidaodilekayissan,48.988691,51.011309,autres,-2.022618,media,0


In [50]:
predict.join(df['cat_autre_struc'].map(_agriculture__)).join(df['nom_struc'])#.dropna('agriculture')

,0,1,categorie,topic,predict_label,classification_kpi,cat_autre_struc,nom_struc
key_main,,,,,,,,
http://YamAgroIndustrie.bf_yameogo_stevearmandleonardwendkouni,76.478304,23.521696,autres,52.956608,agriculture,0,True,Yam agro INDUSTRIE
http://www.aquareignindustry.com_mbouleediango_marcidriss,35.604509,64.395491,autres,-28.790982,media,0,True,Aquareign industry
http://www.congofresh.com_enge_albert,64.193171,35.806829,autres,28.386341,agriculture,0,False,CONGO FOOD COMPANY
http://www.kobo-hub.com_latere_atiyasidonie,79.930454,20.069546,autres,59.860908,agriculture,0,True,Kobo Hub
http://www.leader-manioc-96.websel.net/site%20map.xml_adognon_alidaodilekayissan,48.988691,51.011309,autres,-2.022618,media,0,False,CPA-NEGOCIATION INTERNATIONALE Sarl
http://www.madeintogo.com_wattara_fankjah,64.386611,35.613389,autres,28.773223,agriculture,0,True,FANKFOOD
http://www.senebusiness.com_sene_ablaye,73.139760,26.860240,autres,46.279521,agriculture,0,False,SENE BUSINESS
https://4piment.blogspot.com/_sinare_idrissa,59.044767,40.955233,autres,18.089534,agriculture,0,True,Piment Agro Sylvo Pastorale
https://web.facebook.com/davenbakery_ongmessom_blanche,59.225808,40.774192,NaN,18.451615,agriculture,0,NaN,NaN


# Startup non inscrits en agriculture

In [26]:
df = Chalenge.X
df['corpus'] = df.join(Chalenge.descriptions_trad)[['prez_struc','prez_produit_struc']].agg(sum, axis = 1)
df['type'] = df['corpus'].map(type)
df['corpus'] = df['corpus'].map(vocab)
df['corpus_lemm'] = df['corpus'].map(lemm)
df['collection_counter'] = df['corpus_lemm'].map(col_count)
df = df[df['type'] ==str]

n_components = 2
n_top_words = 20
n_features = 20000
text_corpus = df[df['categorie']!= 'agriculture']['collection_counter']

predict = pandas.DataFrame(model.reverse_nmf(text_corpus.values.tolist(), models) , index =text_corpus.index)
predict = predict.join(df['categorie'])
predict[0] = predict[0]*100
predict[1] = predict[1]*100
predict['topic'] = predict[0]-predict[1]
predict['predict_label'] = predict['topic'].map(get_topic)
predict['classification_kpi'] = predict.apply(lambda x: true_positve(x['categorie'],x['predict_label']), axis= 1)
#predict.sort_values(0, ascending = False)[[0,1,'predict_label']]

In [27]:
#predict.sort_values(0, ascending = False).describe()

In [28]:
#predict[predict['topic'] > 10].sort_values(0, ascending = False).groupby(['categorie','predict_label']).nunique()#.describe(include ='all')

In [29]:
predict[predict['topic']>10].join(chalenge1000.Native().text['temp'])[['categorie','predict_label']]

,categorie,predict_label
key_main,,
http://www.sukuchai.com_kuchengo_suleiman,autres,agriculture
https://www.facebook.com/Community-Action-for-Agriculture-and-Sustainable-Development-466865864068319/_noellifogha_ndze,education,agriculture
http://www.cleanprogb.com%20en%20contruction_etimbibang_confort,finance,agriculture
https://afrikapu.com/_mohamed_nasreenali,ecommerce,agriculture
https://www.facebook.com/TESSA-POWER-SARL-1540172142980252/?ref=bookmarks_ousmane_mahamanlaouali,energie,agriculture
...,...,...
https://solarkoodo.com/_nana_safiatou,energie,agriculture
http://www.agrolimgroceriesuganda.com/_denis_muhereza,ecommerce,agriculture
http://www.afrimart.com.ng/_ojeniyi_olajide,ecommerce,agriculture


In [31]:
a = predict[predict['topic']>10].join(chalenge1000.Native().text['temp'])['temp'].values.tolist()
print(a[0])

Nom du répondant : Kuchengo 
Prénom du répondant : Suleiman 
email : suleykuchengo@yahoo.com 
 
Nom de la structure : SUKU CHAI 
Pays d'origine : Tanzania 
Pays d'activité : Tanzania 
Site internet : http://www.sukuchai.com 
Linkedin : https://www.linkedin.com/in/suleiman-kuchengo-05a370123/?originalSubdomain=tz 
 
Présentation de la structure 
Our main goal is to improve lives in remote, impoverished and neglected rural areas by creating improved access to productive resources as well as meaningful jobs to single mothers, widows, people with disabilities and young girls through tea farming and value addition.
Our approach is based on building an inclusive economy as we are challenging and disrupting the status quo by changing the way marginalized groups and excluded communities were left in the stake of the economy by giving them the necessary skills, tools, and resources to own the economy.
What we do is to empower those marginalized and neglected groups through training and incentiv